# Functions to ease the creation and manipulation of the keybinding tables in the GUI

> This module contains several functions that are used in creating the FastHTML gui binding tables.

In [ ]:
#| default_exp gui_binding_tables

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from monsterui.all import *
from fastcore.test import *

In [ ]:
#| export

def get_modifier_name(db, modifier_id):
    """Safely get the modifier name, returning 'None' if not found"""
    try:
        return next(db.t.modifiers.rows_where("id = ?", [modifier_id]))['name']
    except StopIteration:
        return 'None'

In [ ]:
#| export
def get_key_name(db, key_id):
    """Safely get the key name, returning 'Please select  key' if not found"""
    try:
        return next(db.t.game_keys.rows_where("id = ?", [key_id]))['name']
    except StopIteration:
        return 'Please select a key'

In [ ]:
#| export
def get_action_name(db, action_id):
    """Safely get the action name, returning 'Unknow action' if not fouund"""
    try:
        return next(db.t.actions.rows_where("id = ?", [action_id]))['name']
    except StopIteration:
        return 'Unknown action'

In [ ]:
#| export
def create_binding_table_category(db, game_id, action_category_id, print_layout=False):
    print("Creating binding table for category", action_category_id)
    print("Game ID:", game_id)

    """Helper function to create the bindings table for a given action category"""
    # Get all categories and create lookup dicts
    cat_actions = db.t.actions.rows_where("category_id = ?", [action_category_id])
    action_ids = L(cat_actions).attrgot("id")

    cat_name = L(db.t.categories.rows_where("id = ?", [action_category_id])).attrgot('name')[0]

    # Create table with category groups
    table_heading = Tr(
        Th("  ", style="width:10px;"),
        Th("Action"),
        Th("Key"),
        Th("Modifier"),
        Th("Game native description"),
        Th("Actions")
    )

    if print_layout:
        table_heading = Tr(
            Th("Action"),
            Th("Key (modifier)")
        )

    text_style = TextT.justify

    rows = []
       
    # Add bindings for this category
    action_ids_str = ','.join(str(id) for id in action_ids) 
    # Convert action_ids to string. Converting to a tuple creates a trailing comma with only one element. Which gives invalid SQL syntax.
    where_string = f"game_id = {game_id} AND action_id IN ({action_ids_str}) ORDER BY sort_order"

    def non_tap_modifier(modifier):
        if modifier != "tap":
            return f"   ({modifier})"
        return ""

    for b in db.t.bindings.rows_where(where_string):
        if not print_layout:
            rows.append(Tr(
                Td("⋮⋮", style="width: 10px;"),
                Td(get_action_name(db, b['action_id']),
                    cls=text_style),
                Td(get_key_name(db, b['key_id']),
                    cls=text_style),
                Td(get_modifier_name(db, b['modifier_id']),
                    cls=text_style),
                Td(b['description'],
                    cls=text_style),
                Td(Button("Edit", 
                        hx_get=f"/binding/{b['id']}/edit",
                        hx_target="#bindings-table",
                        cls=ButtonT.secondary),
                    Button("Delete", 
                        hx_delete=f"/binding/{b['id']}/delete",
                        hx_target=f"#form-category-{action_category_id}",
                        cls=ButtonT.destructive),
                    cls="space-x-2"),
                Hidden(name="binding_id", value=b['id']),
                )
            )
        if print_layout:
            rows.append(Tr(
                Td(get_action_name(db, b['action_id']),
                    cls=text_style),
                Td(get_key_name(db, b['key_id']),
                P(non_tap_modifier(get_modifier_name(db, b['modifier_id'])), cls=TextT.italic),
                cls=text_style),
                Td(b['description'], cls=TextT.muted),
                Hidden(name="binding_id", value=b['id']),
                )
            )
    
    return Div(
        P(cat_name, cls=(TextT.lg, TextT.bold, TextT.primary, TextT.center)),
        Form(
            Table(Thead(table_heading),
                  Tbody(*rows,
                        cls='sortable',
                        id=f"sortable-tbody-{action_category_id}"),
                cls=(TableT.hover, TableT.sm, TableT.striped)
            ),
            hx_post=f"/game/{game_id}/reorder_bindings/{action_category_id}",
            hx_trigger="end from:tbody",
            hx_target=f"#form-category-{action_category_id}",
        ),
        id=f"form-category-{action_category_id}"
    )

In [ ]:
#| export
def create_bindings_table(db, game_id):
    """Create tables for all action categories and stack them vertically"""
    # Get all unique category IDs
    categories = db.t.categories.rows_where("id IN (SELECT DISTINCT category_id FROM actions)")
    category_ids = L(categories).attrgot('id')
    
    # Create a table for each category
    tables = [create_binding_table_category(db, game_id, cat_id) for cat_id in category_ids]
    
    # Stack tables in a container div
    return Div(*tables, id="all-bindings-tables")


In [ ]:
#| export
def create_bindings_table_print(db, game_id):

    categories = db.t.categories.rows_where("id IN (SELECT DISTINCT category_id FROM actions)")
    category_ids = L(categories).attrgot('id')

    tables = [create_binding_table_category(db, game_id, cat_id, print_layout=True) for cat_id in category_ids]

    return Card(Grid(
        Card(tables[0]),
        Card(tables[1]),
        Card(*tables[2:]),
        cols=3, cls='gap-12'))

In [ ]:
#| export
def create_actions_table(db):
    """Create a table for all actions"""
    headers = db.t.actions()[0].keys()
    rows = db.t.actions()
    return TableFromDicts(headers, rows, id="actions-table")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()